<a href="https://colab.research.google.com/github/chumpblocckami/AmazonTextClustering/blob/master/ImageScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Scraper su scryfall

In [0]:
from bs4 import BeautifulSoup
import requests 
import time 
import pandas as pd
import random
import shutil
import json

In [0]:
def getAPI(link = "https://scryfall.com/sitemap.xml"):
  xml = requests.get(link).text
  links = xml.split("<url>")
  print("Raw links:",len(links))
  card_links_raw = [x for x in links if "card" in x]
  print("Links to cards:",len(card_links_raw))
  #preprocessing: it's an xml file, with 2 steps we process it as a list of links
  card_links = [x.replace("</loc>","").replace("<loc>","").replace("</url>","").replace("\n","") for x in card_links_raw]
  api_raw = [x.replace("//","//api.") for x in card_links][19:] #useless links
  api = ["/".join(x.split("/")[:3])+search+str(x.split("/")[-1]) for x in api_raw] #get links ready for API
  return api

In [0]:
def getImages(link,path=""):
  try:
    x = requests.get(link).text
    y = json.loads(x)
    images = pd.DataFrame(dict(pd.DataFrame(y["data"])["image_uris"]))[0].tolist()
    c = 0
    for i in images:
      response = requests.get(i, stream=True)
      name = link.split("?q=")[1] + str(c) + '.png'
      with open(path+name, 'wb') as out_file:
          shutil.copyfileobj(response.raw, out_file)
      del response
      c = c+1
  except:
   print("Download of {0} failed! Retry".format(link))

In [0]:
apis = getAPI()
c = 0
for i in apis:
  getImages(i,path="drive/My Drive/mtgDataset/images/")
  time.sleep(1)
  c +1
  if(c%1000):
    print("Downloaded {0} cards! {1} to go...".format(c,len(apis)-c))

COSE DA FINIRE: 
  - BISOGNA CAPIRE COME METTERE LE VARIE CARTE DENTRO AI VARI FOLDER
  - ALCUNE CARTE 'SPECIALI' COME TOKENS O ALTRO NON LE PRENDE (skippiamo)